In [ ]:
!pip install facenet-pytorch
!pip install Pillow

In [ ]:
from inception import InceptionResnetV1
import torch 
resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet.to(device)



# TESTING NN1 (InceptionResnetV1) on clean test set

In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import re
from torchvision import transforms
from utils import load_image_NN1
from utils import get_labels


LABELS = get_labels()

dataset_dir = "test_set_cropped"

correct_predictions = 0
total_images = 0

for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        person_path = os.path.join(dataset_dir, filename)
        img = load_image_NN1(person_path).to(device)
        probs = resnet(img)
        pred_label = LABELS[np.array(probs.cpu().detach().numpy()).argmax()]
        print("Immagine:", filename)
        print("Predizione del modello:", pred_label)
        correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
        print("Etichetta corretta:", correct_label)


        if correct_label in pred_label:
                correct_predictions += 1
                print("OK")
        else:
             with open('predizioni_errate.txt', 'a') as file:
              file.write(filename)
              file.write('\n')

        total_images += 1
        print("-------------------------->")


if total_images == 0:
    print("Nessuna immagine trovata nel dataset.")
else:
    test_accuracy = correct_predictions / total_images
    print('\n')
    print('\n')
    print("Test Accuracy:", test_accuracy)
    print("Numero di predizioni corrette:", correct_predictions)
    print("Numero di immagini processate:", total_images)

# Single immage test

In [ ]:
from utils import load_image_NN1
import os

dataset_dir = "test_set_cropped"
filename = "Andrea_Pirlo_1_face_0.jpg"
person_path = os.path.join(dataset_dir, filename)
img = load_image_NN1(person_path)
probs = resnet(img)
pred_label = LABELS[np.array(probs.detach().numpy()).argmax()]

print('Predicted: ', pred_label)